In [ ]:
!!pip install emoji

# Active transfer learning
pool is a list of data that it hasn't been labled yet.
labeled_data is a list of data that has been labeled

# Setup

In [ ]:
!git clone "https://github.com/parsafarinnia/Active-Learning-for-Rumor-Identification-on-Social-Media"
DATA_PATH = '/content/Active-Learning-for-Rumor-Identification-on-Social-Media'

Cloning into 'sentiment_new_approach'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 60 (delta 16), reused 33 (delta 4), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [ ]:
import tensorflow as tf
import torch
import os
import pickle
import pandas as pd
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import matplotlib.pyplot as plt
% matplotlib inline
import json
import seaborn as sns
from tensorflow import keras
from sklearn.metrics import matthews_corrcoef
!pip install modAL
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Representation

## TweetBERT (Or BERT)

### Data

In [ ]:
all_topics = pickle.load( open( "/content/Active-Learning-for-Rumor-Identification-on-Social-Media/all_topics.p", "rb" ) )
data=pd.DataFrame.from_dict(all_topics[0]).T
data['topic']=0
for i in range(1,9):
  topic = pd.DataFrame.from_dict(all_topics[i]).T
  if i == 3: print('hi')
  topic['topic'] = i
  data = pd.concat([data,topic],axis=0)
map = {"rumours":1,"non-rumours":0}
data=data.replace({'tag':map})
data  

### Model

In [ ]:
from transformers import AutoModel, AutoTokenizer 
# Load BERT TWEET tokenizer and model
bert_tweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer_tweet = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
def tokenizer_func(tokenizer_kind,sentences,labels):
  '''
  inputs:
    tokenizer_kind: is the the tokenizer of choice (normal bert, tweet bert)  
    sentences: train , dev, test
  outputs:
  torchs of 
    ids
    attention_mask
    labels
  '''
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer_kind.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 128,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                          truncation=True,
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  return input_ids, attention_masks ,labels
  # Print sentence 0, now as a list of IDs.


In [ ]:
sentences = data.text.values
train_labels = data.tag.values
input_ids,attention_masks,labels=tokenizer_func(tokenizer_tweet,sentences,train_labels)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
batch_size=128
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
bert_tweet.cuda()
bert_tweet.eval()
# Tracking variables 
sentence_feature=[]
t1 = time.time()
for batch in prediction_dataloader:
  t0 = time.time()  
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      outputs = bert_tweet(b_input_ids, attention_mask=b_input_mask)
      
  sentence_features_slice = outputs[0][:,0,:].cpu().numpy()  
  # Store predictions and true labels
  sentence_feature.append(sentence_features_slice)
  elapsed = format_time(time.time() - t0)
  print("time elapse:",elapsed)
print("full time",format_time(time.time()-t1))
sentence_feature = np.concatenate(sentence_feature, axis=0)
print(sentence_feature.shape)

time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 0:00:02
time elapse: 

In [ ]:

with open('tweet_bert.npy', 'wb') as f:
    np.save(f, sentence_feature)

## Glove word embeding

### Preprocessing

In [ ]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords
import re
import os

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Change "@name" to "Username"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    - change urls to "Link"
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Change "@name" to "Username
    s = re.sub(r'(@.*?)[\s]', 'username', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    # change urls to "Link"
    s = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'link',s)
    
    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np
sentences = np.array([text_preprocessing(text) for text in data.text.values  ])



### Glove



In [ ]:
!wget 'https://nlp.stanford.edu/data/glove.twitter.27B.zip' 

In [ ]:
!unzip "glove.twitter.27B.zip"

In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
glove = loadGloveModel('glove.twitter.27B.50d.txt')

Loading Glove Model
1193514  words loaded!


In [ ]:

data_train = np.zeros(shape=(len(sentences),50),dtype=np.float)

for i in range(len(sentences)):
  tweet = sentences[i]
  tweet = tweet.split(' ')
  glove_word_count = 1
  for word in tweet:
    if word.lower() in glove:
      glove_word_count +=1
      data_train[i,:]+= glove[word]
  data_train[i,:]=data_train[i,:]/(glove_word_count)
features=data_train
#representation Glove

In [ ]:
sentence_feature_GLOVE=features

In [ ]:
sentence_feature_GLOVE.shape

(6425, 100)

In [ ]:
save_reesult(sentence_feature_GLOVE,'GLOVE50.np')